Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.png)

# Register Spark Model and deploy as Webservice

This example shows how to deploy a Webservice in step-by-step fashion:

 1. Register Spark Model
 2. Deploy Spark Model as Webservice

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

### Register Model

You can add tags and descriptions to your Models. Note you need to have a `iris.model` file in the current directory. This model file is generated using [train in spark](../training/train-in-spark/train-in-spark.ipynb) notebook. The below call registers that file as a Model with the same name `iris.model` in the workspace.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model. Note that tags must be alphanumeric.

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="iris.model",
                       model_name="iris.model",
                       tags={'type': "regression"},
                       description="Logistic regression model to predict iris species",
                       workspace=ws)

### Fetch Environment

You can now create and/or use an Environment object when deploying a Webservice. The Environment can have been previously registered with your Workspace, or it will be registered with it as a part of the Webservice deployment.

In this notebook, we will be using 'AzureML-PySpark-MmlSpark-0.15', a curated environment.

More information can be found in our [using environments notebook](../training/using-environments/using-environments.ipynb).

In [ ]:
from azureml.core import Environment
from azureml.core.environment import SparkPackage
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment('my-pyspark-environment')
myenv.docker.base_image = "mcr.microsoft.com/mmlspark/release:0.15"
myenv.inferencing_stack_version = "latest"
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=["azureml-core","azureml-defaults","azureml-telemetry","azureml-train-restclients-hyperdrive","azureml-train-core"], python_version="3.6.2")
myenv.python.conda_dependencies.add_channel("conda-forge")
myenv.spark.packages = [SparkPackage("com.microsoft.ml.spark", "mmlspark_2.11", "0.15"), SparkPackage("com.microsoft.azure", "azure-storage", "2.0.0"), SparkPackage("org.apache.hadoop", "hadoop-azure", "2.7.0")]
myenv.spark.repositories = ["https://mmlspark.azureedge.net/maven"]


## Create Inference Configuration

There is now support for a source directory, you can upload an entire folder from your local machine as dependencies for the Webservice.
Note: in that case, your entry_script is relative path to the source_directory path.

Sample code for using a source directory:

```python
inference_config = InferenceConfig(source_directory="C:/abc",
                                   entry_script="x/y/score.py",
                                   environment=environment)
```

 - source_directory = holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - entry_script = contains logic specific to initializing your model and running predictions
 - environment = An environment object to use for the deployment. Doesn't have to be registered

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

### Deploy Model as Webservice on Azure Container Instance

Note that the service creation can take few minutes.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aci_service_name = 'aciservice1'

try:
    # if you want to get existing service below is the command
    # since aci name needs to be unique in subscription deleting existing aci if any
    # we use aci_service_name to create azure aci
    service = Webservice(ws, name=aci_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

#### Test web service

In [ ]:
import json
test_sample = json.dumps({'features':{'type':1,'values':[4.3,3.0,1.1,0.1]},'label':2.0})

test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = service.run(input_data=test_sample_encoded)
print(prediction)

#### Delete ACI to clean up

In [ ]:
service.delete()

### Model Profiling

You can also take advantage of the profiling feature to estimate CPU and memory requirements for models.

```python
profile = Model.profile(ws, "profilename", [model], inference_config, test_sample)
profile.wait_for_profiling(True)
profiling_results = profile.get_results()
print(profiling_results)
```

### Model Packaging

If you want to build a Docker image that encapsulates your model and its dependencies, you can use the model packaging option. The output image will be pushed to your workspace's ACR.

You must include an Environment object in your inference configuration to use `Model.package()`.

```python
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)  # Or show_output=False to hide the Docker build logs.
package.pull()
```

Instead of a fully-built image, you can also generate a Dockerfile and download all the assets needed to build an image on top of your Environment.

```python
package = Model.package(ws, [model], inference_config, generate_dockerfile=True)
package.wait_for_creation(show_output=True)
package.save("./local_context_dir")
```